In [9]:
%cd ~/google_drive/code/projects/sparse/python
from sparse.core.sparse_dataframe import SparseDataFrame
from sparse.utilities.utils import *

%cd ~/google_drive/code/projects/texture_classifier/python
# %cd /home/ubuntu/texture_classifier/python
import PIL
from sklearn.cluster import KMeans
from scipy.cluster import hierarchy
from scipy.spatial.distance import pdist, squareform
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
import cv2
from pandas.io.pytables import HDFStore
# from plotly import plotly
# from plotly.graph_objs import *

import core.utils
reload(core.utils)
from core.utils import *

import core.image_scanner
reload(core.image_scanner)
from core.image_scanner import ImageScanner

def get_report(y_true, y_pred):
    x = classification_report(y_true, y_pred)
    x = re.sub('avg / total', 'total', x)
    x = map(lambda x: re.split(' +', x), x.split('\n'))
    x = map(lambda x: filter(lambda x: x != '', x), x)
    x = filter(lambda x: x != [], x)
    report = DataFrame(x[1:])
    report.set_index(0, inplace=True)
    report.columns = x[0]
    return report

def _random_scan(item, params):
    return [x for x in ImageScanner(item, **params).random_scan(params['patches'])]

def image_split(source, train=True, test_size=0.2):
    image = PIL.Image.open(source)
    test = list(image.getbbox())
    test[-1] = int(test[-1] * test_size)
    train = list(image.getbbox())
    train[1] = test[-1] + 1
    if train:
        return image.crop(train)
    return image.crop(test)

def _pil_to_opencv(item):
    return cv2.cvtColor(cv2.cvtColor(np.array(item), cv2.COLOR_RGB2BGR), cv2.COLOR_BGR2RGB)

def get_data(info, test_size=0.2):
    def _get_data(info, train, test_size):
        data = info.copy()

        data.source = data.source.apply(lambda x: image_split(x, train, test_size))
        temp = data[['source', 'params']].apply(lambda x: (x['source'], x['params']), axis=1)
        data.source = temp.apply(lambda x: _random_scan(x[0], x[1]))

        def func(item):
            output = DataFrame()
            output['source'] = item[0]
            output['y'] = item[1]
            output['params'] = None
            output.params = output.params.apply(lambda x: item[2])
            return output

        sdf = SparseDataFrame(data)
        data = sdf.merge_columns(['source', 'common_name', 'params'], func=func, new_column='new')
        data = pd.concat(data.new.tolist(), ignore_index=True)

        data['rgb'] = data.source.apply(_pil_to_opencv)
        data['hsv'] = data.rgb.apply(lambda x: cv2.cvtColor(x, cv2.COLOR_RGB2HSV))

        temp = data[['rgb', 'params']].apply(lambda x: (x['rgb'], x['params']), axis=1)
        data['r'] = temp.apply(lambda x: get_channel_histogram(x[0], 'r', **x[1])).apply(lambda x: x.tolist())
        data['g'] = temp.apply(lambda x: get_channel_histogram(x[0], 'g', **x[1])).apply(lambda x: x.tolist())
        data['b'] = temp.apply(lambda x: get_channel_histogram(x[0], 'b', **x[1])).apply(lambda x: x.tolist())

        temp = data[['hsv', 'params']].apply(lambda x: (x['hsv'], x['params']), axis=1)
        data['h'] = temp.apply(lambda x: get_channel_histogram(x[0], 'h', **x[1])).apply(lambda x: x.tolist())
        data['s'] = temp.apply(lambda x: get_channel_histogram(x[0], 's', **x[1])).apply(lambda x: x.tolist())
        data['v'] = temp.apply(lambda x: get_channel_histogram(x[0], 'v', **x[1])).apply(lambda x: x.tolist())

        sdf = SparseDataFrame(data)
        data = sdf.flatten(dtype=list)

        data = data.drop(['source', 'params', 'rgb', 'hsv'], axis=1)

        index = data.index.tolist()
        np.random.shuffle(index)
        data = data.ix[index]
        data.reset_index(drop=True, inplace=True)
        return data
    return _get_data(info, True, test_size), _get_data(info, False, test_size)

/Users/alexbraun/google_drive/code/projects/sparse/python
/Users/alexbraun/google_drive/code/projects/texture_classifier/python


In [18]:
%%snakeviz
source = '/Users/alexbraun/Documents/data/texture_classifier/data/png'
# source = '/home/ubuntu/texture_classifier/data/png'
spec = ['texture', 'image_id', 'image_class', 'common_name', 'pass_', 'source_extension']

info = get_info(source, spec)
info = info[(info.pass_ == 'diffuse') & (info.image_class == 'a')]
info.reset_index(drop=True, inplace=True)

res = 100
params = {
            'min_resolution':   (res, res),
            'max_resolution':   (res, res),
            'patches':                 100,
            'patch_resolution': (res, res),
            'normalize': False,
            'bins': 256
#             'rotation':         'random'
}
info['params'] = None
info.params = info.params.apply(lambda x: params)

# data = get_data(info)
%time train, test = get_data(info, test_size=0.2)

# hdf = HDFStore('/Users/alexbraun/Documents/data/texture_classifier/data/hdf/data.001.hdf')
hdf['train'] = train
hdf['test'] = test
hdf['info'] = info
hdf.close()

CPU times: user 12min 15s, sys: 4min 16s, total: 16min 31s
Wall time: 19min 18s
 
*** Profile stats marshalled to file u'/var/folders/my/jym0xmbx4_g52jkbgll7r1h80000gn/T/tmpJ9Zg6c'. 


In [25]:
# drop = filter(lambda x: True if re.search('y|h|s|v|g|b', x) else False, data.columns.tolist())
# X = data.drop(drop, axis=1)
# X = data.drop('y', axis=1)
# y = data.y

# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)

train_x = train.drop('y', axis=1)
test_x = test.drop('y', axis=1)
train_y = train.y
test_y = test.y

clf = RandomForestClassifier()
params = {
#     'n_estimators':        [10]
#     'criterion':           ['gini'],
#     'max_depth':           [None],
#     'min_samples_split':   [2],
#     'min_samples_leaf':    [1],
#     'max_features':        ['auto'],
#     'max_leaf_nodes':      [None],
#     'bootstrap':           [True],
#     'oob_score':           [False],
#     'n_jobs':              [-1]
#     'random_state':        [42]
#     'verbose':             [0],
#     'min_density':         [None],
#     'compute_importances': [None]
}
grid = GridSearchCV(clf, params, cv=5)
grid.fit(train_x, train_y)
# print(grid.best_score_)
pred = grid.best_estimator_.predict(test_x)
report = get_report(test_y, pred)
report

,precision,recall,f1-score,support
0,,,,
afara-white,0.95,0.98,0.97,100
afrormosia,0.96,0.95,0.95,100
afzelia,0.99,1.00,1.00,100
alder-black,0.93,0.99,0.96,100
alder-red,0.99,1.00,1.00,100
amazakou,0.92,0.98,0.95,100
amazakou-light,0.98,1.00,0.99,100
amburana,0.97,0.99,0.98,100
aningeria,1.00,0.99,0.99,100


In [31]:
x = Series(grid.best_estimator_.feature_importances_, index=train_x.columns)
def func(item):
    for char in list('rgbhsv'):
        if char in item:
            return char
x.rename_axis(func, inplace=True)
x = x.groupby(x.index).sum()
x.sort(ascending=False, inplace=True)
x

/Applications/anaconda/envs/dev/lib/python2.7/site-packages/ipykernel/__main__.py:8: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


s    0.257535
r    0.175271
g    0.169043
v    0.158677
b    0.133155
h    0.106318
dtype: float64